In [56]:
import geemap
import ee
import ipywidgets as widgets
from ipyleaflet import WidgetControl
import datetime as dt
import numpy as np
import os

In [114]:
ee.Authenticate()

KeyboardInterrupt: Interrupted by user

In [57]:
#Mapa de gee

Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [58]:
# CREACION DE WIDGETS

In [59]:
# Creamos un widget para modificar la fecha de inicio
date01_wgt = widgets.DatePicker(
    description='From date:',
    value = dt.date(2021, 1, 1),
    #style={"description_width": "initial"},
    layout=widgets.Layout(width="215px")
)

# Creamos un widget para modificar la fecha de final
date02_wgt = widgets.DatePicker(
    description='To date:',
    value = dt.date(2022, 1,1),
    layout=widgets.Layout(width="215px")
)

#Box de los widgets
date_wgt = widgets.HBox([date01_wgt, date02_wgt])

In [60]:
#Widget con un cuadro de texto para definir el ImageCollection
img_coll_wgt = widgets.Text(
    value='',
    placeholder='e.g. MODIS/006/MOD11A1',
    description='ImageColl:',
    layout=widgets.Layout(width="300px"),
    #style={'description_width': 'initial'},
    disabled=False    
)

In [61]:
#Widget con un cuadro de texto para definir la banda del ImageCollection
band_wgt = widgets.Text(
    value='',
    placeholder='e.g. LST_Day_1km, B4, B5 ',
    description='Band:',
    layout=widgets.Layout(width="300px"),
    #style={"description_width": "initial"},
    disabled=False
)

In [62]:
#Widget cuadro de texto para añadir la paleta
pal = widgets.Textarea(
    value="",
    placeholder="e.g. '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6'",
    description='Palette: ',
    layout=widgets.Layout(width="400px"),
    #style={"description_width": "initial"},
    disabled=False
)

In [63]:
#Widgets max min para el rango de visualizacion
vis_range1 = widgets.FloatText(
    value=0,
    description='VisRange: Min',
    layout=widgets.Layout(width="200px"),
    style = {'description_width': 'initial'},
    disabled=False
    
)

vis_range2 = widgets.FloatText(
    value=0,
    description='Max',
    layout=widgets.Layout(width="200px"),
    
    disabled=False
)

#Box para representacion de los rango de vis.
pal_range_box = widgets.HBox([vis_range1, vis_range2])

In [64]:
#Widget con un cuadro de texto para definir la banda del ImageCollection anterior
layer_name = widgets.Text(
    value='',
    placeholder='e.g: layer_1',
    description='Layer Name:',
    layout=widgets.Layout(width="300px"),
    #style = {'description_width': 'initial'},
    disabled=False
)

In [65]:
#Widget boton para cargar la funcion param()
button = widgets.Button(
    description='Load Parameters', 
    button_style='primary',
    tooltip='Click me'
)

#Widget que utilizaremos en la funcion param()
output = widgets.Output()

#Box con widget button y output
button_box = widgets.VBox([button, output])

In [66]:
#Widget para elegir las imagens disponibles del imageCollection y su output
output_2 = widgets.Output()

image_wgt = widgets.Dropdown(
    options='',
    value=None,
    description="Image:",
    #layout=widgets.Layout(width=widget_width, padding=padding),
    #style={"description_width": "initial"}
)

#Box con image_wgt y output_2
image_wgt_box = widgets.VBox([image_wgt, output_2])

In [67]:
#Widget con el tipo de reduction que se quiere aplicar al imageCollection
reduce_wgt = widgets.Dropdown(
    options=['mean', 'min', 'max', 'median', 'sum'],
    value='mean',
    description='Reduce:',
    layout=widgets.Layout(width="300px"),
    disabled=False,
    #style={"description_width": "initial"}
)

In [68]:
#Widget que define el nombre del archivo de a descargar con las estadisticas
download_name_wgt = widgets.Text(
    value='',
    placeholder='file.csv, .geojson, .kml, .kmz, .shp',
    description='File Name:',
    layout=widgets.Layout(width="300px"),
    style = {'description_width': 'initial'},
    disabled=False
)

Text(value='', description='File Name:', layout=Layout(width='300px'), placeholder='download_file', style=Desc…

In [69]:
#Widget que define el tipo de estadistico que se aplica al archivo de descarga
stat_wgt = widgets.Dropdown(
    options=['MEAN', 'MIN', 'MAX', 'MEDIAN', 'SUM'],
    value='MEAN',
    description='Statistic:',
    layout=widgets.Layout(width="300px"),
    disabled=False,
    #style={"description_width": "initial"}
)

Dropdown(description='Statistic:', layout=Layout(width='300px'), options=('MEAN', 'MIN', 'MAX', 'MEDIAN', 'SUM…

In [70]:
#Widget boton para cargar la funcion get_stats()
button2 = widgets.Button(
    description='Download Stats', 
    button_style='primary',
    tooltip='Click me'
)

output_3 = widgets.Output()

#Box con widget button y output
button2_box = widgets.VBox([button2, output_3])

TypeError: get_stats() takes 0 positional arguments but 1 was given

TypeError: get_stats() takes 0 positional arguments but 1 was given

TypeError: get_stats() takes 0 positional arguments but 1 was given

TypeError: get_stats() takes 0 positional arguments but 1 was given

TypeError: get_stats() takes 0 positional arguments but 1 was given

TypeError: get_stats() takes 0 positional arguments but 1 was given

TypeError: get_stats() takes 0 positional arguments but 1 was given

TypeError: get_stats() takes 0 positional arguments but 1 was given

TypeError: get_stats() takes 0 positional arguments but 1 was given

In [71]:
# FUNCIONES

In [72]:
#Funciones para fecha de inicio y final respectivamente. Devuelven fechas con formato
def timer1():    
    ss = date01_wgt.value
    date01 = "{}-{:02}-{:02}".format(ss.year, ss.month, ss.day)
    return(date01)
    
def timer2():
    ss = date02_wgt.value
    date02 = "{}-{:02}-{:02}".format(ss.year, ss.month, ss.day)
    return(date02)
    
    

In [73]:
#Analiza si el valor de image_wgt ha cambiado, si lo hace cambian las opciones
def handle_dropdwn_click(change):
    
    #Si la banda es mas de una, las lista
    if band_wgt.value == '':
        print('add a valid band to continue')
    else:
        band_val = band_wgt.value.replace(",", "")
        band_val = band_val.split()
    
    #Input teniendo en cuenta la lista de band_val (se modifica respecto a appweb2_v2_15112022)
    img_check = ee.ImageCollection(img_coll_wgt.value)\
        .filterDate(timer1(), timer2())
    
    #Analiza si cambia el valor de img_coll_wgt y si lo hace, carga la lista de imagenes disponibles dentro de la coleccion
    #(se modifica respecto a appweb2_v2_15112022)
    if change['new'] != '':
        img_list = img_check.aggregate_array("system:id").getInfo()
        img_list.insert(0, '')       
        image_wgt.options = img_list
    else:
        print('Add a valid image collection or band')
    
#Analisis de cambios en el parametro img_coll_wgt.value        
img_coll_wgt.observe(handle_dropdwn_click, "value")

In [74]:
#Devuelve los parametros de imageCollection, fecha, banda, reduce y ROI sin cargalo al mapa
def param_val():    
    #creacion de listas si fuera necesario para band_wgt.value
    if band_wgt.value == '':
        print('add a valid band to continue')
    else:
        band_val = band_wgt.value.replace(",", "")
        band_val = band_val.split()                

    #Introducimos la coleccion, introduciomos la banda
    data_load = ee.ImageCollection(img_coll_wgt.value)\
    .select(band_val)\
    .filterDate(timer1(), timer2())       

    #Definimos el estadistico para reducir la imageCollection
    if reduce_wgt.value == 'mean':
        data_load_reducer = data_load.mean()
    elif reduce_wgt.value == 'max':
        data_load_reducer = data_load.max()
    elif reduce_wgt.value == 'min':
        data_load_reducer = data_load.min()
    elif reduce_wgt.value == 'median':
        data_load_reducer = data_load.median()
    elif reduce_wgt.value == 'sum':
        data_load_reducer = data_load.sum()

    #Se determina si es una image o una imagecollection y si tiene ROI o no. Se modifica data_load
    if image_wgt.value == '' and len(Map.draw_features) == 0:
        data_load = ee.ImageCollection(img_coll_wgt.value)\
        .select(band_val)
        data_load = data_load_reducer 
    elif image_wgt.value == '' and len(Map.draw_features) != 0:
        data_load = ee.ImageCollection(img_coll_wgt.value)\
        .select(band_val)
        data_load = data_load_reducer
        data_load = data_load.clip(Map.draw_features[0])
    elif image_wgt.value != '' and len(Map.draw_features) == 0:
        data_load = ee.Image(image_wgt.value)\
        .select(band_val)
    elif image_wgt.value != '' and len(Map.draw_features) != 0:
        data_load = ee.Image(image_wgt.value)\
        .select(band_val)\
        .clip(Map.draw_features[0])

    return(data_load)

In [75]:
#Definimos la funcion param() que cargara los parametros en la aplicacion 

def param(b):
    with output:
        #creacion de listas si fuera necesario para band_wgt.value
        if band_wgt.value == '':
            print('add a valid band to continue')
        else:
            band_val = band_wgt.value.replace(",", "")
            band_val = band_val.split()                
               
        #Introducimos la coleccion, introduciomos la banda
        data_load = ee.ImageCollection(img_coll_wgt.value)\
        .select(band_val)\
        .filterDate(timer1(), timer2())       
        
        #Definimos el estadistico para reducir la imageCollection
        if reduce_wgt.value == 'mean':
            data_load_reducer = data_load.mean()
        elif reduce_wgt.value == 'max':
            data_load_reducer = data_load.max()
        elif reduce_wgt.value == 'min':
            data_load_reducer = data_load.min()
        elif reduce_wgt.value == 'median':
            data_load_reducer = data_load.median()
        elif reduce_wgt.value == 'sum':
            data_load_reducer = data_load.sum()
                    
        #Se determina si es una image o una imagecollection y si tiene ROI o no. Se modifica data_load
        if image_wgt.value == '' and len(Map.draw_features) == 0:
            data_load = ee.ImageCollection(img_coll_wgt.value)\
            .select(band_val)
            data_load = data_load_reducer 
        elif image_wgt.value == '' and len(Map.draw_features) != 0:
            data_load = ee.ImageCollection(img_coll_wgt.value)\
            .select(band_val)
            data_load = data_load_reducer
            data_load = data_load.clip(Map.draw_features[0])
        elif image_wgt.value != '' and len(Map.draw_features) == 0:
            data_load = ee.Image(image_wgt.value)\
            .select(band_val)
        elif image_wgt.value != '' and len(Map.draw_features) != 0:
            data_load = ee.Image(image_wgt.value)\
            .select(band_val)\
            .clip(Map.draw_features[0])
        
        #Comprobación del valor de pal.value, si hay valor, se da formato si no, None
        if pal.value == '':
            pal_val = None
        else:
            pal_val = pal.value.replace("'", "")
            pal_val = pal_val.replace(",", "")
            pal_val = pal_val.split()
                
        band_vis = {
  'min': vis_range1.value, #Añadimos el valor minimo del rango de la paleta
  'max': vis_range2.value, #Añadimos el valor maximo del rango de la paleta
  'palette': pal_val #Se le da valor al parametro paleta teniendo en cuenta el paso anterior
}
                
        #Cargamos todos los parametros a Map.    
        Map.addLayer(data_load, band_vis, layer_name.value)  
        
#Boton que activa la función param()
button.on_click(param)

In [107]:
#obtener estadisticas
def get_stats(b):
    with output_3:
        #Si hay roi, se descarga las estadisticas dentro de este, si no, se imprime mensaje
        if len(Map.draw_features) != 0:
            out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
            out_dem_stats = os.path.join(out_dir, download_name_wgt.value)
            stat_roi = Map.draw_features[0]
            stat_coll_roi = ee.FeatureCollection([stat_roi])
            geemap.zonal_statistics(param_val(), stat_coll_roi, out_dem_stats, statistics_type=(stat_wgt.value), scale=1000)
        else:
            print('add a valid area ROI to the map')

button2.on_click(get_stats)

In [77]:
# FONTEND

In [111]:
#Representamos los widgets para cargar la informacion en el mapa
wgts = [date_wgt, band_wgt, img_coll_wgt, image_wgt_box, reduce_wgt, pal, pal_range_box, layer_name]

input_wgt = widgets.GridBox(wgts, layout=widgets.Layout(grid_template_columns="repeat(3, 570px)", border='3px solid black'))
input_wgt

In [ ]:
#Representamos el boton para cargar los parametros al mapa
button_box

In [112]:
#Representamos los widgets para descargar la informacion estadistica del mapa
download_wgt_list = [download_name_wgt, stat_wgt]
download_wgt = widgets.GridBox(download_wgt_list, layout=widgets.Layout(grid_template_columns="repeat(1, 570px)", border='3px solid black'))
download_wgt

In [ ]:
#Representamos el boton para la descarga de las estadisticas
button2_box

In [81]:
# PRUEBAS

In [ ]:
#Map.user_roi()

In [ ]:
#Map.draw_features

In [ ]:
#a = Map.draw_features[0]

In [ ]:
# collection_no2 = ee.ImageCollection('MODIS/006/MCD15A3H') \
#   .select('Fpar') \
#   .filterDate('2005-09-24', '2005-10-05')\
#   .median()

# coll_clip = collection_no2.clip(a)

# band_viz = {
#   'min': 0,
#   'max': 100,
#   'palette': ['e1e4b4', '999d60', '2ec409', '0a4b06']
# }

# Map.addLayer(coll_clip, band_viz, '1')

In [ ]:
#min(), max(), mean(), median(), sum()

In [ ]:
#Ver la informacion de cada imagen que contiende el imagecollection (collection es image.Collection)
#collection.aggregate_array("system:id").getInfo()